<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_NER_Training_(blank_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy
# !python -m spacy download zh_core_web_sm
# !python -m spacy download zh_core_web_trf

     |████████████████████████████████| 6.4 MB 6.7 MB/s 
     |████████████████████████████████| 10.1 MB 55.6 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 456 kB 51.7 MB/s 
     |████████████████████████████████| 621 kB 33.8 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [39]:
import spacy
import random
from spacy.training.example import Example

# Create a blank "zh" model
nlp = spacy.blank("zh")

# Create a new entity recognizer and add it to the pipeline
ner = nlp.create_pipe("ner")
nlp.add_pipe("ner")

# Add the label "STOCK", "FIN" to the entity recognizer
ner.add_label("STOCK")
ner.add_label("FIN")

TRAINING_DATA = [
  (
    '現時恒指已跌穿重要支持位26500點來看，港股繼續尋底的機會是頗高的。',
    {
        'entities':
        [
         (2,4,'FIN'),(9,12,'FIN'),(21,23,'FIN')
        ]
    }
  ),
  (
    '恆生指數和創業板指數。',
    {
      'entities': 
      [
        (0,4,'FIN'),(5,10,'FIN')          
      ]
    }
  ),  
  (
    '中電、匯控、和黃8月暴升8.5%。',
    {
      'entities': 
      [
        (0,2,'STOCK'),(3,5,'STOCK'),(6,8,'STOCK'),(10,12,'FIN')          
      ]
    }
  ),
  (
    '恆指小雙底9月盼反彈。中電、匯控、和黃8月齊齊跌8.5%金融海嘯後最差。',
    {
      'entities': 
      [
        (0,2,'FIN'),(2,5,'FIN'),(8,10,'FIN'),(11,13,'STOCK'),(14,16,'STOCK'),(17,19,'STOCK')          
      ]
    }
  ),     
  (
    '匯控(00005)將於下周一（2日）公布2021年度中期業績',
    {
        'entities':
        [
          (0,2,'STOCK'),(4,9,'STOCK')
        ]
    }
  ),
  (
    '查看最新的中電股票報價、記錄、新聞及其他重要資訊，助你賣買股票及投資。',
   {
      'entities':
      [
        (21,28,'STOCK')
      ]
   }
  ),
  (
    '美股異動受港股收跌拖累，和黃醫藥(HCM.US)低開低走跌超14%',
    {
        'entities':
        [
         (0,2,'FIN'),(5,7,'FIN'),(7,11,'FIN'),(12,16,'STOCK'),(17,23,'STOCK'),(24,28,'FIN')
        ]
    }
  )
]

# Start the training
nlp.begin_training()

# Loop for n iterations
for itn in range(30):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]

        # Update the model
        # nlp.update(texts, annotations, losses=losses)
        example = []
        for i in range(len(texts)):
          doc = nlp.make_doc(texts[i])
          example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(example,losses=losses)

    print(losses)

{'ner': 169.9149661064148}
{'ner': 113.48558589816093}
{'ner': 41.480406285554636}
{'ner': 38.413625840817986}
{'ner': 69.50898702545237}
{'ner': 66.84218920953572}
{'ner': 36.15499497728888}
{'ner': 28.97237423794104}
{'ner': 32.383636318899846}
{'ner': 26.520248449904997}
{'ner': 20.04466566973774}
{'ner': 13.950912911951136}
{'ner': 13.275791992609939}
{'ner': 60.62641661198313}
{'ner': 37.89636708039751}
{'ner': 17.03949642146722}
{'ner': 10.1799449206043}
{'ner': 3.425763768110445}
{'ner': 0.5598372757727819}
{'ner': 0.022449948781014306}
{'ner': 0.0005771868002182857}
{'ner': 2.790556077962703e-05}
{'ner': 1.3315104599249664e-05}
{'ner': 5.913699434639074e-06}
{'ner': 2.1400140347083678e-07}
{'ner': 1.1227047313796033e-07}
{'ner': 3.1014642149538457e-07}
{'ner': 9.74802766006135e-08}
{'ner': 1.0880845534658967e-07}
{'ner': 8.206366089180893e-08}


In [40]:
texts = [
  '中電在假期前做好帶動，恒指單日暴升961.22點。',
  '受美股收跌拖累和黃跌超2%',
  '智通財經獲悉，7月6日(周二)，和黃醫藥(HCM.US)低開低走'
]

for text in texts:
  doc = nlp(text)
  
  # for ent in doc.ents:
  #     print(ent.text, ent.start_char, ent.end_char, ent.label_)

  from spacy import displacy
  from IPython.core.display import display, HTML

  html = displacy.render(doc,style="ent")
  display(HTML(html))
  print('\n')
